In [1]:
import requests
import json
import numpy as np
import pandas as pd
from unicodedata import normalize
from sqlalchemy import create_engine
import os
import psycopg2
import io
import matplotlib.pyplot as plt
import warnings
import folium
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from google.colab import drive
import plotly.graph_objects as go

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
#Conection to database
host = 'team73test.c4xr0dadx0gz.us-east-2.rds.amazonaws.com'
port = 5432
user = 'postgres'
password = 'ds4a_t73'
database = 'postgres'

In [3]:
connDB = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = connDB.raw_connection()
cur = conn.cursor()

In [4]:
%load_ext sql
%sql postgresql://{user}:{password}@{host}/{database}

'Connected: postgres@postgres'

In [5]:
#Loading Bitcarrier
df = pd.read_sql(''' SELECT * FROM carguebogota.bitcarr ''', conn)

###Adding coordinates and cleaning data

In [6]:
#Data cleaning
df['name_from'] = df['name_from'].replace('KR50,CL13', 'CL13')
df['name_from'] = df['name_from'].replace('CL13,KR50', 'CL13')

df['name_to'] = df['name_to'].replace('CL5A;KR47', 'KR50;KR47')
df['name_to'] = df['name_to'].replace('CL17;KR56', 'KR50;KR56')
df['name_to'] = df['name_to'].replace('KR62;CL5A', 'KR62;KR50')

In [7]:
#Defining direction
West_East = ['KR97;KR90', 'KR123;KR106', 'CL17;KR129', 'KR106;KR97','KR128;KR123','KR128;AV.CARACAS','KR27;KR22','KR22;AV.CARACAS',
              'KR39;KR33','KR33;KR27','KR47;KR39','KR56;KR47','KR65;KR56','KR68;KR65','KR68D;KR68','KR90;KR68D','KR62;KR50','KR50;KR47','KR119;AV.CARACAS',
              'KR24;CARACAS', 'KR69H;KR58','KR76;KR71','KR81;KR76','KR89A;KR81', 'KR58;KR24', 'KR112A;KR102','KR119;KR112A','KR71;KR69H;',
              'KR102;KR96','KR96;KR89A','KR90;KR68','KR128;KR106']
East_West = ['KR90;KR97','KR97;KR106','KR106;KR123','KR129;CL17','KR123;KR128','AV.CARACAS;KR128','KR50;KR56','KR22;KR27','AV.CARACAS;KR22',
              'KR33;KR39','KR39;KR47','KR27;KR33','KR47;KR56','KR56;KR65','KR65;KR68','KR68;KR68D','KR68D;KR90','AV.CARACAS;KR119','CARACAS;KR24',
              'KR69H;KR71','KR76;KR81','KR81;KR89A','KR24;KR58','KR58;KR69H','KR112A;KR119','KR71;KR76' ,'KR102;KR112A''KR96;KR102','KR89A;KR96',
              'KR68;KR90','KR106;KR128']
df['orientation'] = 'west_east'
df['orientation'] = df['orientation'].where(df['name_to'].isin(West_East), 'east_west')

In [8]:
#Defining coordenates 
coordenadas1 = {'KR97;KR90': ([-74.14853144099999, 4.666731241846797],
  [-74.13958359200001, 4.66322384184687]),
 'KR90;KR97': ([-74.13959365, 4.663407966846895],
  [-74.14852004199997, 4.6669153668467676]),
 'KR123;KR106': ([-74.15966864400002, 4.686716340846148],
  [-74.152940322, 4.673972885846544]),

 #'CL17;KR129': ([[-74.13959365, 4.663407966846895],
 #  [-74.16347621800003, 4.690046288846024]]),
 
 'KR106;KR97': ([-74.15295172200001, 4.67388500184656],
  [-74.14872456, 4.666804757846783]),
 'KR97;KR106': ([-74.148692374, 4.666987879846787],
  [-74.15278006, 4.6739705468465464]),
 'KR106;KR123': ([-74.15278006, 4.674109557846537],
  [-74.159474854, 4.686962612846106]),

 #'KR129;CL17': ([[-74.16369079499998, 4.689864509846029],
 # [-74.13958359200001, 4.66322384184687]]),

 'KR128;KR123': ([-74.16369079499998, 4.689864509846029],
  [-74.15974258300002, 4.686870486846144]),
 'KR123;KR128': ([-74.15954946400001, 4.6870308808461125],
  [-74.16347621800003, 4.690046288846024]),

# 'AV.CARACAS;KR128': ([[-74.07990931400002, 4.603861481848834],
#   [-74.16347621800003, 4.690046288846024]]),
# 'KR128;AV.CARACAS': ([[-74.16369079499998, 4.689864509846029],
#   [-74.079995145, 4.603668985848824]]),
# 'CL17;KR56': ([-74.10533665600002, 4.629727653847998],
#  [-74.11004661499999, 4.630438792847955]),

 'KR27;KR22': ([-74.089361419, 4.6118393248485665],
  [-74.08700107499999, 4.609711195848607]),
 'KR22;KR27': ([-74.08688305800001, 4.610010631848636],
  [-74.08914684199999, 4.612010430848545]),
 'AV.CARACAS;KR22': ([-74.07990931400002, 4.603861481848834],
  [-74.08670066799999, 4.609860913848646]),
 'KR22;AV.CARACAS': ([-74.08690451500001, 4.609721889848635],
  [-74.079995145, 4.603668985848824]),
 'KR33;KR27': ([-74.09449516699999, 4.616376281848395],
  [-74.08945261399998, 4.611906162848576]),
 'KR33;KR39': ([-74.09424840399998, 4.616611550848413],
  [-74.09915148200002, 4.620974700848282]),
 'KR39;KR33': ([-74.09959136399999, 4.620857066848242],
  [-74.09456490399998, 4.616403016848387]),
 'KR39;KR47': ([-74.09923194800001, 4.621022822848241],
  [-74.104105522, 4.625377924848134]),
 'KR47;KR39': ([-74.10428254800001, 4.625107904848125],
  [-74.09968524099997, 4.620937271848254]),
 'KR27;KR33': ([-74.08931313900001, 4.612173515848538],
  [-74.094216217, 4.616568774848427]),
 'KR56;KR47': ([-74.11038725499998, 4.630064508847984],
  [-74.104368378, 4.62517206784811]),
 'KR47;KR56': ([-74.104135026, 4.625396639848137],
  [-74.11012976299997, 4.630337201847981]),
 'KR56;KR65': ([-74.11014652699998, 4.630466195847943],
  [-74.114566808, 4.634631425847818]),
 'KR65;KR56': ([-74.11476730300001, 4.634342025847845],
  [-74.11041139500003, 4.630289079847973]),
 'KR65;KR68': ([-74.11475657400001, 4.634737694847827],
  [-74.11796449600001, 4.637304190847732]),
 'KR68;KR65': ([-74.118191142, 4.637183886847728],
  [-74.11501004299998, 4.63459065584782]),
 'KR68;KR68D': ([-74.118313183, 4.637643716847724],
  [-74.12185369899998, 4.640146039847636]),
 'KR68D;KR68': ([-74.12215410599998, 4.639846616847637],
  [-74.11861359, 4.637344292847741]),
 'KR68D;KR90': ([-74.12217556399997, 4.640381300847623],
  [-74.13942753200001, 4.663313498846907]),
 'KR90;KR68D': ([-74.139486541, 4.663121018846908],
  [-74.12230967400001, 4.640007022847652]),

# 'KR62;CL5A': ([-74.11781429199999, 4.6281529858480095],
#  [-74.11165593999999, 4.620763494848274]),
# 'CL5A;KR47': ([-74.11148427900002, 4.620592390848272],
#  [-74.10448907799997, 4.625212169848124]),
# 'AV.CARACAS;KR119': ([[-74.06101013400001, 4.666237280846801],
#   [-74.12464319600002, 4.726235508844826]]),
# 'KR119;AV.CARACAS': ([[-74.12494360400001, 4.7259788918448455],
#   [-74.06105874899998, 4.665849147846814]]),

 'CARACAS;KR24': ([-74.06101013400001, 4.666237280846801],
  [-74.06341121399998, 4.669238909846679]),
 'KR24;CARACAS': ([-74.06340534700001, 4.668773753846683],
  [-74.06105874899998, 4.665849147846814]),
 'KR69H;KR58': ([-74.08261298100001, 4.688132254846106],
  [-74.07448052299998, 4.678401610846421]),
 'KR69H;KR71': ([-74.08237426400001, 4.6885385868460485],
  [-74.087202241, 4.694376904845891]),
 'KR76;KR71': ([-74.09331767700002, 4.697349509845761],
  [-74.087717225, 4.694291361845883]),
 'KR76;KR81': ([-74.09317015599999, 4.697841377845783],
  [-74.09722565599998, 4.700300715845692]),
 'KR81;KR76': ([-74.09735440200001, 4.699915776845694],
  [-74.09355639400002, 4.697477822845764]),
 'KR81;KR89A': ([-74.09729002900002, 4.700364872845687],
  [-74.103298177, 4.70438533584552]),
 'KR89A;KR81': ([-74.103577127, 4.704064554845563],
  [-74.09754752100002, 4.700001318845711]),
 'KR24;KR58': ([-74.063365449, 4.669312425846669],
  [-74.07400845500001, 4.678487155846406]),
 'KR58;KR24': ([-74.07428740400002, 4.678230521846378],
  [-74.06358002600001, 4.668948855846692]),
 'KR58;KR69H': ([-74.07422303099997, 4.678658244846394],
  [-74.082398404, 4.688431657846084]),
 'KR119;KR112A': ([-74.12494360400001, 4.7259788918448455],
  [-74.11968647399999, 4.719392366845072]),
 'KR102;KR112A': ([-74.11365150400002, 4.711634943845336],
  [-74.11929487200001, 4.719419098845057]),
 'KR112A;KR102': ([-74.11955236400001, 4.719280096845088],
  [-74.11384462299998, 4.7113890138453405]),
 'KR112A;KR119': ([-74.11932705800001, 4.719568792845059],
  [-74.12464319600002, 4.726235508844826]),
 'KR71;KR69H;': ([-74.08748119, 4.694077504845876],
  [-74.08278195999998, 4.688367499846097]),
 'KR71;KR76': ([-74.08737390200002, 4.694633532845872],
  [-74.092845608, 4.69767029284575]),
 'KR96;KR102': ([-74.10937069800002, 4.707550365845468],
  [-74.11338328300002, 4.711421091845338]),
 'KR102;KR96': ([-74.113748063, 4.711185854845323],
  [-74.10967110600001, 4.707186815845446]),
 'KR89A;KR96': ([-74.10340546499998, 4.704492262845564],
  [-74.10924195199999, 4.707400668845434]),
 'KR96;KR89A': ([-74.10947798699999, 4.707079888845457],
  [-74.10368441499998, 4.704064554845563]),
 'KR90;KR68': ([-74.139486541, 4.663121018846908],
  [-74.12230967400001, 4.640007022847652]),
 'KR68;KR90': ([-74.12217556399997, 4.640381300847623],
  [-74.13942753200001, 4.663313498846907])}


In [9]:
#Add coordinates as new column to db
coordenadas = {}
for i in coordenadas1.keys():
  np.flip(coordenadas1[i][0])
  coordenadas.update( {i: (np.flip(coordenadas1[i][0]) , np.flip(coordenadas1[i][1]))})
df['coordinates']= df['name_to'].map(coordenadas)

In [10]:
#Translating weekday
df_bitcar = df[df['year'] == 2019]
dias = ['Domingo', 'Jueves', 'Viernes', 'Lunes', 'Sabado', 'Martes','Miercoles']
days = ['Sunday','Thursday','Friday','Monday','Saturday','Tuesday','Wednesday']
df_bitcar['weekday'] = df_bitcar['weekday'].replace(dias,days)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



###Split df into CL13 and CL80

In [11]:
#The corridors of interest are CL13 and CL80
df_CL13 = df_bitcar[df_bitcar['name_from'].isin(['DIAG16', 'CL17', 'CL13'])]
df_CL80 = df_bitcar[df_bitcar['name_from'] == 'CL80']
#Dataframe with both avenues
df_bitcarrier = df_bitcar[df_bitcar['name_from'].isin(['DIAG16', 'CL17', 'CL13','CL80'])]

### *Average speed data for Dash*

In [12]:
#Generating info for Dash
avg_speed_dash = df_bitcarrier[[ 'name_from','name_to','month','weekday', 'hour', 'orientation','average_speed']]

#Including Peak hours restrictions
restriction ={0: ' No',1: ' No',2: ' No',3: ' No',4: ' No',5: ' No',6: ' Yes',7: ' Yes',8: ' Yes',9: ' No',10: ' No',11: ' No',12: 
              ' No',13: ' No',14: ' No',15: ' No',16: ' No',17: ' Yes',18: ' Yes',19: ' Yes',20: ' No',21: ' No',22: ' No',23: ' No',}
avg_speed_dash['Restriction'] = avg_speed_dash['hour'].map(restriction)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
#Dataframe to generate info for dash for both corridors

avg_speed_dash = avg_speed_dash.groupby(['name_from', 'name_to', 'month', 'weekday', 
                                         'hour', 'orientation','Restriction'])['average_speed'].mean().reset_index()

In [14]:
#Plot for Average speed per weekday
avg_speed_weekday = avg_speed_dash[['month','average_speed','weekday']].groupby(['month','weekday']).mean().reset_index()
days_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig = px.bar(avg_speed_weekday, 
             x="weekday", 
             y="average_speed", 
             animation_frame="month", 
             animation_group="weekday",
             color_discrete_sequence= ['rgb(158,202,225)'],
             text = 'average_speed',
             category_orders=dict(weekday=days_order),
             title = 'Average speed by weekday',
             labels = { 'average_speed': 'Average speed [km/h]',
                       'weekday': 'Weekday'})
fig.update_traces(texttemplate = '%{text:.1f} km/h', 
                  textposition = 'outside')
fig.update_layout(font_family = 'Times New Roman',
                  title = {'x':0.5})
fig['layout'].pop("updatemenus")
fig.update_yaxes(range=[0, 40])
fig.show()

In [15]:
#Plot for Average speed per hour
avg_speed_hour = avg_speed_dash.groupby(['month','hour','orientation','Restriction']).mean().reset_index()
fig = px.bar(avg_speed_hour, 
             x="hour", y="average_speed", 
             animation_frame="month", 
             animation_group="hour", 
             title = 'Average speed by hour',
             color = 'Restriction', 
             color_discrete_map = {' No':'rgb(158,202,225)',
                                   ' Yes':'rgb(8,48,107)'},
             text = 'average_speed',
             labels = {'average_speed': 'Average speed [km/h]',
                       'hour': 'Hour'})      
fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.update_layout(font_family = 'Times New Roman',
                  title = {'x':0.5})
fig.update_traces(texttemplate = '%{text:.1f}', 
                  textposition = 'outside')
fig.update_yaxes(range=[0, 55])
fig.update_layout(xaxis = dict(tickmode = 'linear',
                               tick0 = 0,
                               dtick = 1))
fig.show()

# **Map Plotly CALLE 13**

In [16]:
#Splitting by direction
df_CL13_ew = df_CL13[df_CL13['orientation'] == 'east_west']
df_CL13_we = df_CL13[df_CL13['orientation'] == 'west_east']

df_CL80_ew = df_CL80[df_CL80['orientation'] == 'east_west']
df_CL80_we = df_CL80[df_CL80['orientation'] == 'west_east']

In [17]:
#Obtaining the av. speed of the corridor
CL13_ew_vel = df_CL13_ew[['name_to','average_speed','coordinates']].groupby(['name_to']).mean().reset_index()
CL13_we_vel = df_CL13_we[['name_to','average_speed','coordinates']].groupby(['name_to']).mean().reset_index()

CL13_ew_vel['coordinates']= CL13_ew_vel['name_to'].map(coordenadas)
CL13_we_vel['coordinates']= CL13_we_vel['name_to'].map(coordenadas)

CL13_ew_vel = CL13_ew_vel.dropna()
CL13_we_vel = CL13_we_vel.dropna()

In [18]:
#Function to obtain coordinates for each segment of the corridor
def tramo(df):
  tramo_from_lat = []
  tramo_from_long = []
  tramo_to_lat = []
  tramo_to_long = []
  for i in range(df.shape[0]):
    s_from,s_to = df['coordinates'].iloc[i]
    tramo_from_lat = np.append(tramo_from_lat,s_from[0])
    tramo_from_long = np.append(tramo_from_long,s_from[1])
    tramo_to_lat = np.append(tramo_to_lat,s_to[0])
    tramo_to_long = np.append(tramo_to_long,s_to[1])
  return tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long

In [19]:
#Function to include a new column with separate coordinates
def add_tramo (tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long,df):
  df['tramo_from_lat'] = tramo_from_lat
  df['tramo_from_long'] = tramo_from_long
  df['tramo_to_lat'] = tramo_to_lat
  df['tramo_to_long'] = tramo_to_long
  return df

In [20]:
#Defining order of segments in DF

road_order_ew = {'AV.CARACAS;KR22': 1, 'KR106;KR123':13, 'KR123;KR128':14, 'KR22;KR27':2,
              'KR27;KR33':3, 'KR33;KR39':4, 'KR39;KR47':5, 'KR47;KR56':6, 'KR56;KR65':7,
              'KR65;KR68':8, 'KR68;KR68D':9, 'KR68D;KR90':10, 'KR90;KR97':11, 'KR97;KR106':12}

CL13_ew_vel['order'] = CL13_ew_vel['name_to'].map(road_order_ew)
CL13_ew_vel = CL13_ew_vel.sort_values('order')

road_order_we = {'KR106;KR97':3,'KR123;KR106':2, 'KR128;KR123':1, 'KR22;AV.CARACAS':14,'KR27;KR22':13, 
                 'KR33;KR27':12, 'KR39;KR33':11, 'KR47;KR39':10, 'KR56;KR47':9,'KR65;KR56':8, 
                 'KR68;KR65':7, 'KR68D;KR68':6, 'KR90;KR68D':5, 'KR97;KR90':4}

CL13_we_vel['order'] = CL13_we_vel['name_to'].map(road_order_we)
CL13_we_vel = CL13_we_vel.sort_values('order')
CL13_we_vel = CL13_we_vel.dropna()

In [21]:
tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long = tramo(CL13_ew_vel)
CL13_ew_vel = add_tramo(tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long,CL13_ew_vel)

tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long = tramo(CL13_we_vel)
CL13_we_vel = add_tramo(tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long,CL13_we_vel)
CL13_ew_vel = CL13_ew_vel.dropna()

In [22]:
#Colors for specific speed
#20-25 #25-30 #30-35 #>35 #<20
r1 = '#d73027';r2 = '#fdae61';r3 = '#a6d96a';r4 = '#1a9850';r5 = '#480909'

color_ew = [r1,r2,r3,r4,r3,r1,r3,r2,r3,r3,r4,r2,r3,r1]
CL13_ew_vel['color'] = color_ew

color_we = [r2,r4,r3,r4,r4,r2,r3,r2,r4,r3,r3,r2,r4,r1]
CL13_we_vel['color'] = color_we

In [23]:
#Generating map with avg speed per segment of road cl13 East-West
fig = go.Figure()
for i in range(CL13_ew_vel.shape[0]):
  lat1 = CL13_ew_vel[['tramo_from_lat','tramo_to_lat']].iloc[i].to_numpy()
  long1 = CL13_ew_vel[['tramo_from_long','tramo_to_long']].iloc[i].to_numpy()
  line_color = CL13_ew_vel['color'].iloc[i]
  fig.add_trace(go.Scattermapbox(
      mode = "markers+lines",
      lon = long1,
      lat = lat1,
      line=dict(width=10, 
                color= line_color),
      marker=dict(
            size=4,
            showscale=True,
            colorscale='rdylgn',
            cmin=20,
            cmax=50)))
fig.update_layout(showlegend=False,
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -74.12, 'lat': 4.65},
        'style': "stamen-terrain",
        'zoom': 11.9})
fig.show()

In [24]:
#Generating map with avg speed per segment of road cl13 West-East
fig = go.Figure()

for i in range(CL13_we_vel.shape[0]):

  lat1 = CL13_we_vel[['tramo_from_lat','tramo_to_lat']].iloc[i].to_numpy()
  long1 = CL13_we_vel[['tramo_from_long','tramo_to_long']].iloc[i].to_numpy()
  line_color = CL13_we_vel['color'].iloc[i]
  fig.add_trace(go.Scattermapbox(
      mode = "markers+lines",
      lon = long1,
      lat = lat1,
      line=dict(width=10, 
                color= line_color),
      marker=dict(
            size=4,
            showscale=True,
            colorscale='rdylgn',
            cmin=20,
            cmax=50)))
fig.update_layout(showlegend=False,
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -74.12, 'lat': 4.65},
        'style': "stamen-terrain",
        'zoom': 11.9})
fig.show()

# **Map Plotly CALLE 80**

In [25]:
#Obtaining the av. speed of the corridor
CL80_ew_vel = df_CL80_ew[['name_to','average_speed','coordinates']].groupby(['name_to']).mean().reset_index()
CL80_we_vel = df_CL80_we[['name_to','average_speed','coordinates']].groupby(['name_to']).mean().reset_index()

CL80_ew_vel['coordinates']= CL80_ew_vel['name_to'].map(coordenadas)
CL80_we_vel['coordinates']= CL80_we_vel['name_to'].map(coordenadas)

CL80_ew_vel = CL80_ew_vel.dropna()
CL80_we_vel = CL80_we_vel.dropna()

In [26]:
tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long = tramo(CL80_ew_vel)
CL80_ew_vel = add_tramo(tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long,CL80_ew_vel)

tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long = tramo(CL80_we_vel)
CL80_we_vel = add_tramo(tramo_from_lat,tramo_from_long, tramo_to_lat,tramo_to_long,CL80_we_vel)

In [27]:
#Defining order of segments in DF
road_order_we = {'KR102;KR96':3, 'KR112A;KR102':2, 'KR119;KR112A':1, 'KR24;CARACAS':11,
                 'KR58;KR24':10, 'KR69H;KR58':9, 'KR71;KR69H;':8, 'KR76;KR71':7, 
                 'KR81;KR76':6,'KR89A;KR81':5, 'KR96;KR89A':4}

CL80_we_vel['order'] = CL80_we_vel['name_to'].map(road_order_we)
CL80_we_vel = CL80_we_vel.sort_values('order')

road_order_ew = {'CARACAS;KR24':1,'KR102;KR112A':10, 'KR112A;KR119':11, 'KR24;KR58':2,
                 'KR58;KR69H':3, 'KR69H;KR71':4, 'KR71;KR76':5, 'KR76;KR81':6, 'KR81;KR89A':7,
                 'KR89A;KR96':8, 'KR96;KR102':9}

CL80_ew_vel['order'] = CL80_ew_vel['name_to'].map(road_order_ew)
CL80_ew_vel = CL80_ew_vel.sort_values('order')

In [28]:
#Colors for specific speed
color_ew = [r3,r4,r4,r3,r2,r4,r3,r1,r1,r3,r4]
CL80_ew_vel['color'] = color_ew

color_we = [r4,r2,r3,r2,r4,r2,r3,r3,r4,r3,r3]
CL80_we_vel['color'] = color_we

In [29]:
#Generating map with avg speed per segment of road cl80 East-West
fig = go.Figure()
for i in range(CL80_ew_vel.shape[0]):

  lat1 = CL80_ew_vel[['tramo_from_lat','tramo_to_lat']].iloc[i].to_numpy()
  long1 = CL80_ew_vel[['tramo_from_long','tramo_to_long']].iloc[i].to_numpy()
  line_color = CL80_ew_vel['color'].iloc[i]
  fig.add_trace(go.Scattermapbox(
      mode = "markers+lines",
      lon = long1,
      lat = lat1,
      line=dict(width=10, 
                color= line_color),
      marker=dict(
            size=4,
            showscale=True,
            colorscale='rdylgn',
            cmin=20,
            cmax=50)))
fig.update_layout(showlegend=False,
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -74.09, 'lat': 4.7},
        'style': "stamen-terrain",
        'zoom': 12.3})
fig.show()

In [30]:
#Generating map with avg speed per segment of road cl80 West-East
fig = go.Figure()

for i in range(CL80_we_vel.shape[0]):

  lat1 = CL80_we_vel[['tramo_from_lat','tramo_to_lat']].iloc[i].to_numpy()
  long1 = CL80_we_vel[['tramo_from_long','tramo_to_long']].iloc[i].to_numpy()
  line_color = CL80_we_vel['color'].iloc[i]
  fig.add_trace(go.Scattermapbox(
      mode = "markers+lines",
      lon = long1,
      lat = lat1,
      line=dict(width=10, 
                color= line_color),
      marker=dict(
            size=4,
            showscale=True,
            colorscale='rdylgn',
            cmin=20,
            cmax=50)))
fig.update_layout(showlegend=False,
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -74.09, 'lat': 4.7},
        'style': "stamen-terrain",
        'zoom': 12.3})
fig.show()